In [1]:
import pymongo as mng
from pymongo import MongoClient
import pprint

### Вместо Предисловия :)
Для удобства работы развернул mongoDB на локальном компьютере.
Создал `DB=products` и `collection=products`.

И добавил информацию о 4 товарах.

In [2]:
myclient = mng.MongoClient("mongodb://localhost:27017/")
mydb = myclient["products"]
mycol = mydb["products"]

In [3]:
myquery = { "product_category_name": { "$gt": "" }}
myquery = {}


In [4]:
mydoc = mycol.find(myquery)

In [5]:
for x in mydoc:
    pprint.pprint(x)

{'_id': ObjectId('5dd1c3ebf5c2b82ff82c2669'),
 'product_category_name': 'Бритвы',
 'product_name': 'Бритва Braun XXL',
 'product_price': Decimal128('1000.10'),
 'product_unit_available': 100}
{'_id': ObjectId('5dd1c520a368412ff812a8f2'),
 'product_category_name': 'Бритвы',
 'product_name': 'Бритва Gilette Mini',
 'product_price': Decimal128('525.15'),
 'product_unit_available': 10}
{'_id': ObjectId('5dd1c555a368412ff812a8f3'),
 'product_category_name': 'Батарейки ААА',
 'product_name': 'Батарейки Ikea AAA 16 шт.',
 'product_price': Decimal128('325.99'),
 'product_unit_available': 11}
{'_id': ObjectId('5dd1c574a368412ff812a8f4'),
 'product_category_name': 'Батарейки ААА',
 'product_name': 'Батарейки PKCell AAA 32 шт.',
 'product_price': Decimal128('3699'),
 'product_unit_available': 588}


#### Считаем остаточную стоимость товаров

In [6]:
pipeline = [
    {
        "$project": {
            "_id": "$product_category_name",
            "multiply": {"$multiply": [ {"$sum": "$product_price"}, {"$sum": "$product_unit_available"} ]}
        }
    },
    {
        "$group": {
            "_id": "$_id",
            "avaiable_in_stock": {"$sum": "$multiply"}
          }
    }
]


print("Остаточная стоимость товаров в каждой категории:")
pprint.pprint(list(mydb.products.aggregate(pipeline)))



Остаточная стоимость товаров в каждой категории:
[{'_id': 'Бритвы', 'avaiable_in_stock': Decimal128('105261.50')},
 {'_id': 'Батарейки ААА', 'avaiable_in_stock': Decimal128('2178597.89')}]


#### Уменьшаем количество товаров на "единицу"
Я не понял нужно ли результат записывать обратно в БД, поэтому дополнительных манипуляций делать не стал.

In [7]:
pipeline2 = [
    {
        "$project": {
            "_id": "$_id",
            "product_unit_available": {"$sum": ["$product_unit_available",-1]},
            'product_category_name': 1,
            'product_name': 1,
            'product_price': 1
 
        }
    }
]

In [8]:
print("Уменьшаем количество товаров на 1")
pprint.pprint(list(mydb.products.aggregate(pipeline2)))

Уменьшаем количество товаров на 1
[{'_id': ObjectId('5dd1c3ebf5c2b82ff82c2669'),
  'product_category_name': 'Бритвы',
  'product_name': 'Бритва Braun XXL',
  'product_price': Decimal128('1000.10'),
  'product_unit_available': 99},
 {'_id': ObjectId('5dd1c520a368412ff812a8f2'),
  'product_category_name': 'Бритвы',
  'product_name': 'Бритва Gilette Mini',
  'product_price': Decimal128('525.15'),
  'product_unit_available': 9},
 {'_id': ObjectId('5dd1c555a368412ff812a8f3'),
  'product_category_name': 'Батарейки ААА',
  'product_name': 'Батарейки Ikea AAA 16 шт.',
  'product_price': Decimal128('325.99'),
  'product_unit_available': 10},
 {'_id': ObjectId('5dd1c574a368412ff812a8f4'),
  'product_category_name': 'Батарейки ААА',
  'product_name': 'Батарейки PKCell AAA 32 шт.',
  'product_price': Decimal128('3699'),
  'product_unit_available': 587}]


#### Выводим два самых дорогих товара

In [9]:
pipeline3 = [
    {
        "$match":{
            
        }
    },
    {
         "$sort": {"product_price": -1}
    },
    {
        "$limit": 2
    }
]

In [10]:
print("TOP-2 самых дорогих товара:")
pprint.pprint(list(mydb.products.aggregate(pipeline3)))

TOP-2 самых дорогих товара:
[{'_id': ObjectId('5dd1c574a368412ff812a8f4'),
  'product_category_name': 'Батарейки ААА',
  'product_name': 'Батарейки PKCell AAA 32 шт.',
  'product_price': Decimal128('3699'),
  'product_unit_available': 588},
 {'_id': ObjectId('5dd1c3ebf5c2b82ff82c2669'),
  'product_category_name': 'Бритвы',
  'product_name': 'Бритва Braun XXL',
  'product_price': Decimal128('1000.10'),
  'product_unit_available': 100}]
